# MNIST Basic Approach (Softmax)

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
type(mnist)

In [ ]:
mnist.train.num_examples,mnist.test.num_examples,mnist.validation.num_examples

In [ ]:
mnist.train.images.shape

In [ ]:
single_image = mnist.train.images[1].reshape(28,28)

In [ ]:
plt.imshow(single_image, cmap='gist_gray') # show image as gray

In [ ]:
single_image.min(), single_image.max() # note range is 0 to 1 so it's already been normalized

In [ ]:
plt.hist(single_image.reshape(28*28,1),bins=20,);

In [ ]:
# Placeholders
x = tf.placeholder(tf.float32,shape=[None,784])

In [ ]:
# Variables
 # zeros are probably not the best thing to do. there is tf.random
W = tf.Variable(tf.zeros([784,10])) # weights: 784 px and 10 labels
b = tf.Variable(tf.zeros([10])) # bias

In [ ]:
# Create graph operations
y = tf.matmul(x,W) + b # could also be called Z

In [ ]:
# Loss function
y_true = tf.placeholder(tf.float32,[None,10])

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y))

In [ ]:
# Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5)
train = optimizer.minimize(cross_entropy)

In [ ]:
# Create Session
init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for step in range(1000):
        # mnist has a method to provide batches, this isn't always provided
        # Lots of time spent in cleaning, reshaping, and formatting data
        batch_x, batch_y = mnist.train.next_batch(100)
        sess.run(train, feed_dict={x:batch_x,y_true:batch_y})
        
    # Evaluate the model
    # argmax gives highest entry point
    # What index position is y the greatest e.g. what is the highest probability
    correct_predition = tf.equal(tf.argmax(y,1), tf.argmax(y_true,1))  # returns a list of booleans
    
    # e.g. [True,False,True, ...]
    # Idx position of highest prob therefore because one-hot it returns the spot wehre it's true
    # we need to convert list of bools to vector of 0/1
    accuracy = tf.reduce_mean(tf.cast(correct_predition, tf.float32))
    
    # argmax: 
    # PREDICTED=[3, 4] TRUE=[3,9] (e.g. because sometimes 4's look like 9)
    # tf.eq gives [True, False]
    # then we cast the [T,F] to [1.0,0.0]
    # we then take the mean of above array = 0.5 == 50% right
    print(sess.run(accuracy, feed_dict={x:mnist.test.images, y_true: mnist.test.labels}))

91% seems good but the best models now get 99.7% accuracy